In [ ]:
# visualize_benchmark.ipynb
import pandas as pd
import matplotlib.pyplot as plt

# Charger les résultats
df = pd.read_csv('results.csv')

# Remplacer les "timeout" par une valeur indicative (ex: 300 secondes)
df['duration'] = df['duration'].replace('timeout', 300).astype(float)

# Extraire le nom simple des fichiers pour l'axe X
df['benchmark_name'] = df['benchmark_file'].apply(lambda x: x.split('/')[-1])

# Créer un graphique
plt.figure(figsize=(14, 6))
for algo in df['algorithm'].unique():
    subset = df[df['algorithm'] == algo]
    plt.plot(subset['benchmark_name'], subset['duration'], marker='o', label=algo.upper())

plt.xlabel('Benchmarks')
plt.ylabel('Temps d\'exécution (secondes)')
plt.title('Comparaison des algorithmes sur différents benchmarks')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
